# Reinforcement learning with Cross-entropy method optimization (CEM)

CEM performs a policy search with exploration directly in parameter space. A collection of sample policies are sampled from a distribution and tried through experiments. A new, weighted distribution is fit to the sampled policy parameters, where the reward achieved by the policy determines its weight. This way, policies that achieved high rewards are more likely to be sampled next time. This procedure is iterated until performance is adequate.

We focus on Gaussian distributions in parameter space, and illustrate two methods of fitting
new weighted distributions: rank-based and Boltzmann-based. The rank-based fit fits and unweighted Gaussian to the top fraction of policies, whereas the Boltzmann-based strategy assigns a weights of $\exp(r_i)$ to policy $i$, where $r_i$ is the reward achieved by that policy.

For further introduction, see [Levine 2017 (slides)](http://rll.berkeley.edu/deeprlcourse/f17docs/lecture_7_advanced_q_learning.pdf)

This tutorial is available as an [iJulia notebook](https://github.com/baggepinnen/baggepinnen.github.io/blob/master/cem.ipynb)

We start out by importing some packages. If it's your first time, you might have to install some packages using the commands in the initial comment.
OpenAI gym is installed with instructions available at https://gym.openai.com/docs/

In [ ]:
# Pkg.clone("https://github.com/tbreloff/MetaPkg.jl")
# using MetaPkg
# meta_add("MetaRL")
# Pkg.add("Plots")
# Pkg.add("BasisFunctionExpansions")
# Pkg.add("ValueHistories")

using OpenAIGym, ValueHistories, Plots

If you want to perform plotting in the loop, the following function helps keeping the plot clean.

In [ ]:
default(size=(1200,800)) # Set the default plot size
function update_plot!(p; max_history = 10, attribute = :markercolor)
    num_series = length(p.series_list)
    if num_series > 1
        if num_series > max_history
            deleteat!(p.series_list,1:num_series-max_history)
        end
        for i = 1:min(max_history, num_series)-1
            alpha = 1-2/max_history
            c = p[i][attribute]
            b = alpha*c.b + (1-alpha)*0.5
            g = alpha*c.g + (1-alpha)*0.5
            r = alpha*c.r + (1-alpha)*0.5
            a = alpha*c.alpha
            p[i][attribute] = RGBA(r,g,b,a)
        end
    end
end

Next, we define an environment from the OpenAIGym framework, we'll use the [cartpole environment](https://gym.openai.com/envs/CartPole-v0/) and a function that runs an entire episode.

In [ ]:
env = GymEnv("CartPole-v0")

function collect_reward(ep)
    r = Vector{Float64}(0)
    for (ss,aa,rr,ss1) in ep
        push!(r,rr)
    end
    r
end

We also define a linear policy object that is a linear combination of the states.

In [ ]:
struct Policy <: AbstractPolicy
    θ::Vector{Float64}
end

(π::Policy)(s) = π.θ's + 0.5 > 0.5 ? 1 :  0 # policy function

Reinforce.action(π::Policy, r, s, A) = π(s)

num_params  = length(env.state)
num_epochs  = 15
num_samples = 25
Σ           = 0.2eye(num_params) # Initial noise chol(covariance)
μ           = zeros(num_params) # Initial policy mean
const π     = Policy(zeros(num_params)) # π is now our policy object

We have now arrived at the main algorithm. We wrap it in a function for the Julia JIT complier to have it run faster.

In [ ]:
function fit_new_distribution_weighted(θs, w)
    weights = ProbabilityWeights(exp.(w))
    μ       = mean(θs, weights, 2)
    Σ       = cov(θs, weights, 2, corrected=true)
    Σ       = chol(Symmetric(Σ + 1e-5I))'
    μ, Σ
end

function fit_new_distribution_rank(θs, w)
    fraction = 2
    p        = sortperm(w)
    indicies = p[end-end÷fraction:end]
    μ        = mean(θs[:,indicies], 2)
    Σ        = cov(θs[:,indicies], 2)
    Σ        = chol(Symmetric(Σ))'
    μ, Σ
end


gr() # Set GR as plot backend, try pyplot() if GR is not working
function CME(π, μ, Σ, num_epochs, num_samples; plotting=true)
    plotting && (fig = plot(layout=2, show=true))
    reward_history = ValueHistories.History(Float64)
    for i = 1:num_epochs
        θs = Σ*randn(num_params, num_samples) .+ μ # Sample new policies
        weights = zeros(num_samples)
        for n = 1:num_samples # Collect N trajectories for Expectation estimation
            π.θ       .= θs[:,n] # Set policy parameters
            ep         = Episode(env, π)
            s,a,r,s1   = collect_reward(ep)
            weights[n] = ep.total_reward
            push!(reward_history, (i-1)*num_samples+n, ep.total_reward)
        end
        μ, Σ = fit_new_distribution(θs, weights)

        # Printing and plotting
        println("Epoch: $i, reward: $(reward_history.values[end])")
        if plotting
            plot!(reward_history, subplot=1, show=false)
            scatter!(θs, subplot=2, c=:red, title="Policy parameters", show=false, legend=false)
            update_plot!(fig[1], max_history=1, attribute=:linecolor)
            update_plot!(fig[2], max_history=num_samples, attribute=:linecolor)
            gui(fig)
        end
    end
    plot(reward_history, title="Rewards", xlabel="Episodes", show=true)
end

We now call our function, with the two different distribution-refit strategies:

In [ ]:
fit_new_distribution = fit_new_distribution_weighted
srand(1);
CME(π, μ, Σ, num_epochs, num_samples; plotting=true)

In [ ]:
fit_new_distribution = fit_new_distribution_rank
srand(1);
CME(π, μ, Σ, num_epochs, num_samples; plotting=true)